In [2]:
# mount it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Загрузка саугументированного датасета

In [3]:
!wget https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py
!chmod +x ya.py
!./ya.py https://disk.yandex.ru/d/JI4FdHjXQmQ8MQ ./

--2022-11-06 05:25:58--  https://gist.githubusercontent.com/Yegorov/dc61c42aa4e89e139cd8248f59af6b3e/raw/20ac954e202fe6a038c2b4bb476703c02fe0df87/ya.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 723 [text/plain]
Saving to: ‘ya.py’

ya.py               100%[===================>]     723  --.-KB/s    in 0s      

2022-11-06 05:25:58 (36.3 MB/s) - ‘ya.py’ saved [723/723]

--2022-11-06 05:25:59--  https://downloader.disk.yandex.ru/disk/7f1e771de93febfe8f1946ccd0115368824686acc0e4661e07d152592786a6c5/63677da7/axnq710K2FiMN5Ul9lpzKl1J8Y3KUbwA_bBu_jCLAsTrg20DS9gKU5kzs3aJG9kVro6fSeNjXTZshr5SLxw0iw%3D%3D?uid=0&filename=dataset.zip&disposition=attachment&hash=uvYxvbXu1vRR0WNmntrkisz67eieISObnEDN8/jf/z657Qxbr89uqORdnxMs8ct7q/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit

In [4]:
!ls

dataset.zip  drive  sample_data  ya.py


In [5]:
!unzip dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: augumentation/images/8048p9.jpg  
  inflating: augumentation/images/8048p13.jpg  
  inflating: augumentation/images/8048p16.jpg  
  inflating: augumentation/images/8048p20.jpg  
  inflating: augumentation/images/8048p22.jpg  
  inflating: augumentation/images/8048p31.jpg  
  inflating: augumentation/images/8048a3.jpg  
  inflating: augumentation/images/8048a4.jpg  
  inflating: augumentation/images/8048a6.jpg  
  inflating: augumentation/images/8048a14.jpg  
  inflating: augumentation/images/8048a17.jpg  
  inflating: augumentation/images/8048a19.jpg  
  inflating: augumentation/images/8048a20.jpg  
  inflating: augumentation/images/8048a27.jpg  
  inflating: augumentation/images/8103p0.jpg  
  inflating: augumentation/images/8103p1.jpg  
  inflating: augumentation/images/8103p2.jpg  
  inflating: augumentation/images/8103p3.jpg  
  inflating: augumentation/images/8103p12.jpg  
  inflating: augumentation/i

In [6]:
!ls

augumentation  dataset.zip  drive  sample_data	ya.py


In [7]:
!rm dataset.zip

# Создание набора тестовых и тренироваочных даных


In [8]:
import json
import os
import shutil
from pathlib import Path

import albumentations as A
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image

In [9]:
def get_files(files_dir: Path, suff: str = 'txt'):
    return sorted(Path(files_dir).glob(f'*.{suff}'))

In [10]:
aug_dir = Path('augumentation')
crop_image = aug_dir / 'images'
crop_labels = aug_dir / 'labels'

In [11]:
crop_image_list = get_files(crop_image, 'jpg')
crop_labels_list = get_files(crop_labels)


In [12]:
crop_labels_list = [Path(label_path).stem for label_path in crop_labels_list]
#crop_labels_list

In [13]:
dataset_df = pd.DataFrame(columns=['image', 'label', 'image_path', 'label_path'])

In [14]:
dataset_df['image_path'] = crop_image_list

In [15]:
dataset_df.head(3)

,image,label,image_path,label_path
0,NaN,NaN,augumentation/images/3444a0.jpg,NaN
1,NaN,NaN,augumentation/images/3444a1.jpg,NaN
2,NaN,NaN,augumentation/images/3444a10.jpg,NaN


In [16]:
def path_to_name(row: pd.Series) -> pd.Series:
    image = Path(row['image_path'])
    row['image'] = image.name
    if image.stem in crop_labels_list:
        row['label'] = f'{image.stem}.txt'
        row['label_path'] = aug_dir / 'labels' /row['label']
    return row

In [17]:
dataset_df.apply(path_to_name, axis=1)

,image,label,image_path,label_path
0,3444a0.jpg,NaN,augumentation/images/3444a0.jpg,NaN
1,3444a1.jpg,NaN,augumentation/images/3444a1.jpg,NaN
2,3444a10.jpg,NaN,augumentation/images/3444a10.jpg,NaN
3,3444a11.jpg,NaN,augumentation/images/3444a11.jpg,NaN
4,3444a12.jpg,NaN,augumentation/images/3444a12.jpg,NaN
...,...,...,...,...
3765,8507p5.jpg,8507p5.txt,augumentation/images/8507p5.jpg,augumentation/labels/8507p5.txt
3766,8507p6.jpg,8507p6.txt,augumentation/images/8507p6.jpg,augumentation/labels/8507p6.txt
3767,8507p7.jpg,8507p7.txt,augumentation/images/8507p7.jpg,augumentation/labels/8507p7.txt
3768,8507p8.jpg,8507p8.txt,augumentation/images/8507p8.jpg,augumentation/labels/8507p8.txt


In [18]:
dataset_df.head(3)

,image,label,image_path,label_path
0,3444a0.jpg,NaN,augumentation/images/3444a0.jpg,NaN
1,3444a1.jpg,NaN,augumentation/images/3444a1.jpg,NaN
2,3444a10.jpg,NaN,augumentation/images/3444a10.jpg,NaN


In [19]:
label_df = dataset_df[dataset_df['label'].notna()].copy()
non_label_df = dataset_df[dataset_df['label'].isna()].copy()

In [20]:
label_df.head(2)

,image,label,image_path,label_path
30,3444p0.jpg,3444p0.txt,augumentation/images/3444p0.jpg,augumentation/labels/3444p0.txt
31,3444p1.jpg,3444p1.txt,augumentation/images/3444p1.jpg,augumentation/labels/3444p1.txt


In [21]:
dataset_df.shape[0] - label_df.shape[0] - non_label_df.shape[0]

0

In [22]:
def train_validate_test_split(df: pd.DataFrame, train_percent: float = 0.6, validate_percent: float = 0.4, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [23]:
train_df_1, valid_df_1, _ = train_validate_test_split(label_df.reset_index(drop=True))

In [24]:
train_df_2, valid_df_2, _ = train_validate_test_split(non_label_df.reset_index(drop=True))

In [25]:
train_df = pd.concat([train_df_1, train_df_2])
valid_df = pd.concat([valid_df_1, valid_df_2])

In [26]:
dataset_df.shape, valid_df.shape, train_df.shape

((3770, 4), (1507, 4), (2261, 4))

In [27]:
11595+17393

28988

In [28]:
valid_df.head(3)

,image,label,image_path,label_path
1087,6561p33.jpg,6561p33.txt,augumentation/images/6561p33.jpg,augumentation/labels/6561p33.txt
186,4153p19.jpg,4153p19.txt,augumentation/images/4153p19.jpg,augumentation/labels/4153p19.txt
341,4604p3.jpg,4604p3.txt,augumentation/images/4604p3.jpg,augumentation/labels/4604p3.txt


In [29]:
def move_file(short_name: str, target_dir: str) -> None:
    image_file = aug_dir / 'images' / f'{short_name}.jpg'
    new_image = aug_dir / 'images' / target_dir / f'{short_name}.jpg'
    shutil.move(image_file, new_image)
    
    label_file = aug_dir / 'labels' / f'{short_name}.txt'
    new_label = aug_dir / 'labels' / target_dir / f'{short_name}.txt'
    #print(label_file)
    #print(new_label)
    #print(label_file.is_file())
    if label_file.is_file():
        #print('ok')
        shutil.move(label_file, new_label)


In [30]:
def move_handler(row, target_dir):
    name = Path(row['image_path']).stem
    move_file(str(name), target_dir)
    return row

In [31]:
valid_df.apply(move_handler, axis=1, args=('test',))

,image,label,image_path,label_path
1087,6561p33.jpg,6561p33.txt,augumentation/images/6561p33.jpg,augumentation/labels/6561p33.txt
186,4153p19.jpg,4153p19.txt,augumentation/images/4153p19.jpg,augumentation/labels/4153p19.txt
341,4604p3.jpg,4604p3.txt,augumentation/images/4604p3.jpg,augumentation/labels/4604p3.txt
519,5065p3.jpg,5065p3.txt,augumentation/images/5065p3.jpg,augumentation/labels/5065p3.txt
1166,6741p0.jpg,6741p0.txt,augumentation/images/6741p0.jpg,augumentation/labels/6741p0.txt
...,...,...,...,...
1078,6908a19.jpg,NaN,augumentation/images/6908a19.jpg,NaN
654,6105a1.jpg,NaN,augumentation/images/6105a1.jpg,NaN
1456,8103a11.jpg,NaN,augumentation/images/8103a11.jpg,NaN
1397,8020a1.jpg,NaN,augumentation/images/8020a1.jpg,NaN


In [32]:
train_df.apply(move_handler, axis=1, args=('train',))

,image,label,image_path,label_path
394,4871p11.jpg,4871p11.txt,augumentation/images/4871p11.jpg,augumentation/labels/4871p11.txt
1544,7577p31.jpg,7577p31.txt,augumentation/images/7577p31.jpg,augumentation/labels/7577p31.txt
1643,7872p26.jpg,7872p26.txt,augumentation/images/7872p26.jpg,augumentation/labels/7872p26.txt
455,4888p4.jpg,4888p4.txt,augumentation/images/4888p4.jpg,augumentation/labels/4888p4.txt
1399,7049p27.jpg,7049p27.txt,augumentation/images/7049p27.jpg,augumentation/labels/7049p27.txt
...,...,...,...,...
747,6219a12.jpg,NaN,augumentation/images/6219a12.jpg,NaN
1416,8020a27.jpg,NaN,augumentation/images/8020a27.jpg,NaN
1108,6911a2.jpg,NaN,augumentation/images/6911a2.jpg,NaN
729,6153a23.jpg,NaN,augumentation/images/6153a23.jpg,NaN


In [33]:
!ls augumentation/images

5191a3.jpg  6911p32.jpg  test  train


# Обучение модели

In [34]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 14483, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 14483 (delta 16), reused 28 (delta 9), pack-reused 14446
Receiving objects: 100% (14483/14483), 13.47 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (9989/9989), done.
/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 12.4 MB/s 


In [35]:
!ls data

Argoverse.yaml	  GlobalWheat2020.yaml	Objects365.yaml  VOC.yaml
coco128-seg.yaml  hyps			scripts		 xView.yaml
coco128.yaml	  ImageNet.yaml		SKU-110K.yaml
coco.yaml	  images		VisDrone.yaml


In [36]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/import/forest1.yaml data/forest.yaml

In [37]:
!mkdir train train/images train/labels

In [38]:
!mv /content/augumentation/images/train train/images
!mv /content/augumentation/images/test train/images
!mv /content/augumentation/labels/train/ train/labels
!mv /content/augumentation/labels/test train/labels

In [39]:
!ls train/images/train

3444a0.jpg   4604a23.jpg  5680a23.jpg  6368p9.jpg   7012a9.jpg	 8048a14.jpg
3444a10.jpg  4604a24.jpg  5680a25.jpg  6478a0.jpg   7012p10.jpg  8048a17.jpg
3444a13.jpg  4604a26.jpg  5680a26.jpg  6478a10.jpg  7012p13.jpg  8048a18.jpg
3444a14.jpg  4604a27.jpg  5680a27.jpg  6478a11.jpg  7012p14.jpg  8048a19.jpg
3444a17.jpg  4604a28.jpg  5680a29.jpg  6478a12.jpg  7012p17.jpg  8048a1.jpg
3444a18.jpg  4604a29.jpg  5680a3.jpg   6478a13.jpg  7012p18.jpg  8048a21.jpg
3444a19.jpg  4604a3.jpg   5680a4.jpg   6478a14.jpg  7012p20.jpg  8048a22.jpg
3444a1.jpg   4604a6.jpg   5680a9.jpg   6478a16.jpg  7012p21.jpg  8048a26.jpg
3444a20.jpg  4604a9.jpg   5680p10.jpg  6478a17.jpg  7012p24.jpg  8048a27.jpg
3444a21.jpg  4604p0.jpg   5680p11.jpg  6478a19.jpg  7012p25.jpg  8048a3.jpg
3444a24.jpg  4604p11.jpg  5680p12.jpg  6478a20.jpg  7012p28.jpg  8048a7.jpg
3444a25.jpg  4604p13.jpg  5680p13.jpg  6478a22.jpg  7012p29.jpg  8048p11.jpg
3444a29.jpg  4604p15.jpg  5680p16.jpg  6478a24.jpg  7012p31.jpg  8048p12.jpg
344

In [ ]:
#!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_4/best.pt best.pt

cp: cannot stat '/content/drive/MyDrive/colab_export/omsk_forest/export/model_4/best.pt': No such file or directory


In [40]:
!ls runs/train

ls: cannot access 'runs/train': No such file or directory


In [ ]:
!rm -r runs/train/exp

In [ ]:
!python train.py --batch 18 --epochs 5 --data forest.yaml --weights yolov5m6.pt
!cp -r runs/train/exp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/

In [ ]:
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/exp/weights/best.pt best.pt

In [ ]:
!python train.py --batch 18 --epochs 5 --data forest.yaml --weights best.pt
!cp -r runs/train/exp2 /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/

In [41]:
#!rm best.pt
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/exp2/weights/best.pt best.pt

In [42]:
!python train.py --batch 18 --epochs 5 --data forest.yaml --weights best.pt
!cp -r runs/train/exp3 /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/

train: weights=best.pt, cfg=, data=forest.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=5, batch_size=18, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-226-gfde7758 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degr

In [43]:
!cp -r runs/train/exp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/

In [44]:
!rm best.pt
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/exp/weights/best.pt best.pt

In [ ]:
!python train.py --batch 18 --epochs 50 --data forest.yaml --weights best.pt --project /content/drive/MyDrive/colab_export/omsk_forest/export/model_5_1
#!cp -r runs/train/exp4 /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/

train: weights=best.pt, cfg=, data=forest.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=18, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/colab_export/omsk_forest/export/model_5_1, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-226-gfde7758 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0

In [47]:
!ls

benchmarks.py	 data	     models	       segment	       utils
best.pt		 detect.py   __pycache__       setup.cfg       val.py
classify	 export.py   README.md	       train
content		 hubconf.py  requirements.txt  train.py
CONTRIBUTING.md  LICENSE     runs	       tutorial.ipynb


In [ ]:
!rm best.pt
!cp /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/exp4/weights/best.pt best.pt

In [ ]:
!python train.py --batch 18 --epochs 5 --data forest.yaml --weights best.pt
!cp -r runs/train/exp5 /content/drive/MyDrive/colab_export/omsk_forest/export/model_5/